In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pymongo
from selenium import webdriver
from pprint import pprint
import pandas as pd

In [2]:
# Setup configuration variables to enable Splinter to interact with browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
news_url ="https://mars.nasa.gov/news/"
browser.visit(news_url)
news_html = browser.html
news_soup = BeautifulSoup(news_html, "html.parser")
#pprint (news_soup)

In [34]:
# get the div with grid of news articles
gridList = news_soup.find('div', class_='react_grid_list grid_list_container')

# get the first title
try :
    news_title = gridList.find('div', class_='content_title').text
    pprint(news_title)
except :
    news_title = None

# get the first title description
try :
    news_p = gridList.find('div', class_='rollover_description_inner').text
    pprint(news_p)
except :
    news_p = None


"NASA's Perseverance Rover Is Midway to Mars "
('Sometimes half measures can be a good thing – especially on a journey this '
 "long. The agency's latest rover only has about 146 million miles left to "
 'reach its destination.')


In [6]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages'
browser.visit(jpl_url)
image_html = browser.html
image_soup = BeautifulSoup(image_html,"html.parser")
#pprint(image_soup)

In [7]:
# get the carousel_item

carousel_item = image_soup.find("article", "carousel_item")
#pprint (carousel_item)

try :
    featured_image_title, = image_soup.find("h1", "media_feature_title")
except :
    featured_image_title = None

try :
    #get the footer a element of the article
    footer_a_item = carousel_item.find("a", "button fancybox")
    #pprint(f"Atag : {footer_a_item.attrs}")
    
    # get the imageurl from 
    img_url = footer_a_item['data-fancybox-href']
    
    substring = "/spaceimages"
    featured_image_url = jpl_url + img_url.replace(substring, '')
    pprint (featured_image_url)
    
except Exception as ex:
    pprint(ex)
    featured_image_url = None

 


'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17470_ip.jpg'


In [8]:
space_url = 'https://space-facts.com/mars/'
browser.visit(space_url)
space_html = browser.html
space_soup = BeautifulSoup(space_html,"html.parser")
##pprint(space_soup)

In [27]:
#reference :- https://pythonprogramminglanguage.com/web-scraping-with-pandas-and-beautifulsoup/
# get the table element from the page. 
table_item  = space_soup.find("table", "tablepress tablepress-id-p-mars")

df = pd.read_html(str(table_item))

#print (df[0].columns)
df = df[0].rename(columns = {0 : 'Description', 1 : 'Mars'})

df = df.set_index('Description')
#pprint (df)
mars_fact_table = df.to_html( classes="table table-striped")

pprint(mars_fact_table)

Int64Index([0, 1], dtype='int64')
                                               Mars
Description                                        
Equatorial Diameter:                       6,792 km
Polar Diameter:                            6,752 km
Mass:                 6.39 × 10^23 kg (0.11 Earths)
Moons:                          2 (Phobos & Deimos)
Orbit Distance:            227,943,824 km (1.38 AU)
Orbit Period:                  687 days (1.9 years)
Surface Temperature:                   -87 to -5 °C
First Record:                     2nd millennium BC
Recorded By:                   Egyptian astronomers
('<table border="1" class="dataframe table table-striped">\n'
 '  <thead>\n'
 '    <tr style="text-align: right;">\n'
 '      <th></th>\n'
 '      <th>Mars</th>\n'
 '    </tr>\n'
 '    <tr>\n'
 '      <th>Description</th>\n'
 '      <th></th>\n'
 '    </tr>\n'
 '  </thead>\n'
 '  <tbody>\n'
 '    <tr>\n'
 '      <th>Equatorial Diameter:</th>\n'
 '      <td>6,792 km</td>\n'
 '    </tr>\n'
 ' 

In [10]:
astro_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(astro_url)
astro_html = browser.html
astro_soup = BeautifulSoup(astro_html,"html.parser")
#pprint(astro_soup)

In [11]:
# get the mars image urls

hemisphere_image_urls  = []
try :
    
    base_url = "https://astrogeology.usgs.gov"
    
    # get the mars image items
    mars_items = astro_soup.find_all("div", "item")    

    # iterate through the mars image list
    for item in mars_items :
        # get the title for image
        #pprint(item)
        title = item.find('h3').text
        #url for the image 
        item_url =  item.find("a")["href"]
        link_url = base_url + item_url

        # open the page in the browser
        browser.visit(link_url)
        page_html = browser.html
        page_soup = BeautifulSoup(page_html,"html.parser")
        #pprint (page_soup)
        image_item = page_soup.find('div', 'downloads')
        #pprint(image_item)
        image_url = image_item.find('a')['href']    
        
        mars_dict = { "title": title, "img_url": image_url}
        pprint(mars_dict)
        hemisphere_image_urls.append(mars_dict)

except Exception as ex:
    pprint(ex)
    
    

{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'title': 'Cerberus Hemisphere Enhanced'}
{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'title': 'Schiaparelli Hemisphere Enhanced'}
{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'title': 'Syrtis Major Hemisphere Enhanced'}
{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
 'title': 'Valles Marineris Hemisphere Enhanced'}


In [12]:
browser.quit()

In [35]:
# build the dictionary object
mars_dict = {
    'news_title' : news_title,
    'news_p': news_p,
    'featured_image_Title': featured_image_title,
    'featured_image_url': featured_image_url,
    'mars_fact_table': mars_fact_table,
    'hemisphere_image_urls': hemisphere_image_urls
    }
pprint(mars_dict)

{'featured_image_Title': '\n'
                         "\t\t\t\t  Titan's Northern Lakes: Salt "
                         'Flats?\t\t\t\t',
 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17470_ip.jpg',
 'hemisphere_image_urls': [{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
                            'title': 'Cerberus Hemisphere Enhanced'},
                           {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
                            'title': 'Schiaparelli Hemisphere Enhanced'},
                           {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
                            'title': 'Syrtis Major Hemisphere Enhanced'},
                           {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/

In [36]:
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.marsExpedition_db
collection = db.mars 
collection.drop()

#insert the dictionary the collection
collection.insert_one(mars_dict)

In [37]:
# read from mongo

mars_data = collection.find()

for data in mars_data :
    print(data)

{'_id': ObjectId('5f9b834ff1e945b1c84bb7ba'), 'news_title': "NASA's Perseverance Rover Is Midway to Mars ", 'news_p': "Sometimes half measures can be a good thing – especially on a journey this long. The agency's latest rover only has about 146 million miles left to reach its destination.", 'featured_image_Title': "\n\t\t\t\t  Titan's Northern Lakes: Salt Flats?\t\t\t\t", 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA17470_ip.jpg', 'mars_fact_table': '<table border="1" class="dataframe table table-striped">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n     